## Module to perform DATA DISCOVERY in DATA LAKES

## IMPORTS ##

In [ ]:
##################################
# Created: Aug 13, 2023
# Updated: Sep 3, 2023 16:45
# Location: Budapest, Hungary
##################################

import warnings
warnings.filterwarnings("ignore")

import time
import re
import os

import random
from itertools import combinations

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.lines import Line2D

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

from scipy.linalg import cholesky

from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing as mp

import networkx as nx
import networkx.algorithms.community as nx_comm

from discovery_lsh_netw import Node
from discovery_lsh_netw import Edge
from discovery_lsh_netw import create_edge_key
from discovery_lsh_netw import create_netw

import pickle

from csv_schema_inference import csv_schema_inference

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
all_stopwords = stopwords.words('english')

my_random = random.Random(10000)

## CODE FROM https://doi.org/10.1007/s10994-020-05882-8 ##

In [ ]:
"""
The SCD algorithm, Skrlj and Kralj, 2019 (2020 accepted).
"""
import tqdm
import scipy.sparse as sp
import scipy as spy
from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
from itertools import product
import operator

import logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S')
logging.getLogger().setLevel(logging.INFO)


class SCD_obj:
    """
    The main SCD object.
    """
    def __init__(self,
                 input_graph,
                 verbose=True,
                 node_names=None,
                 input_type="sparse_matrix",
                 device="cpu"):
        """
        Initiator class
        """
        if input_type == "networkx":
            node_names = list(input_graph.nodes())
            input_graph = nx.to_scipy_sparse_matrix(input_graph)
        self.verbose = verbose
        self.device = device
        self.all_scores = None
        self.node_names = node_names
        self.cluster_quality = {}
        self.default_parameters = {
            "clustering_scheme": "hierarchical",
            "max_com_num": "100",
            "verbose": False,
            "sparisfy": True,
            "parallel_step": 6,
            "prob_threshold": 0.0005,
            "community_range": [1, 3, 5, 7, 11, 20, 40, 50, 100, 200, 300],
            "fine_range": 3,
            "lag_threshold": 10,
            "num_important": 10000
        }
        self.input_graph = input_graph
        self.quality_scores = {}

    def page_rank_kernel(self, index_row):
        """
        a kernel for parallel PPRS execution
        param: index of a node.
        output: PPR vector
        """

        pr = self.PPRS([index_row],
                       epsilon=1e-6,
                       max_steps=100000,
                       damping=0.90,
                       spread_step=10,
                       spread_percent=0.1,
                       try_shrink=True)
        norm = np.linalg.norm(pr, 2)
        if norm > 0:
            pr = pr / np.linalg.norm(pr, 2)
            return (index_row, pr)
        else:
            return (index_row, np.zeros(self.normalized.shape[1]))

    def sparse_pr(self, max_iter=10000, tol=1.0e-6, alpha=0.85):
        """
        Sparse personalized pagerank.
        """

        N = self.input_graph.shape[1]
        nodelist = np.arange(self.input_graph.shape[1])
        S = spy.array(self.input_graph.sum(axis=1)).flatten()
        S[S != 0] = 1.0 / S[S != 0]
        Q = sp.spdiags(S.T, 0, *self.input_graph.shape, format='csr')
        M = Q * self.input_graph
        x = spy.repeat(1.0 / N, N)
        p = spy.repeat(1.0 / N, N)
        dangling_weights = p
        is_dangling = spy.where(S == 0)[0]
        for _ in range(max_iter):
            xlast = x
            x = alpha * (x * M + sum(x[is_dangling]) * dangling_weights) + \
                (1 - alpha) * p
            err = spy.absolute(x - xlast).sum()
            if err < N * tol:
                return dict(zip(nodelist, map(float, x)))

    def stochastic_normalization(self, matrix=None, return_matrix=False):
        """
        Perform stochastic normalization
        """

        if matrix is None:
            matrix = self.input_graph.tolil()
        try:
            matrix.setdiag(0)
        except TypeError as te:
            matrix.setdiag(np.zeros(matrix.shape[0]))
        matrix = matrix.tocsr()
        d = matrix.sum(axis=1).getA1()
        nzs = np.where(d > 0)
        k = 1 / d[nzs]
        matrix = (sp.diags(k, 0).tocsc().dot(matrix)).transpose()
        if return_matrix:
            return matrix
        else:
            self.normalized = matrix

    def modularity(self, communities, weight='weight'):
        """
        Classic computation of modularity.
        """

        G = nx.from_scipy_sparse_matrix(self.input_graph)
        multigraph = G.is_multigraph()
        directed = G.is_directed()
        m = G.size(weight=weight)
        if directed:
            out_degree = dict(G.out_degree(weight=weight))
            in_degree = dict(G.in_degree(weight=weight))
            norm = 1 / m
        else:
            out_degree = dict(G.degree(weight=weight))
            in_degree = out_degree
            norm = 1 / (2 * m)

        def val(u, v):
            try:
                if multigraph:
                    w = sum(d.get(weight, 1) for k, d in G[u][v].items())
                else:
                    w = G[u][v].get(weight, 1)
            except KeyError:
                w = 0
            # Double count self-loops if the graph is undirected.
            if u == v and not directed:
                w *= 2
            return w - in_degree[u] * out_degree[v] * norm

        Q = np.sum(
            val(u, v) for c in communities for u, v in product(c, repeat=2))

        return Q * norm

    def PPRS(self,
             start_nodes,
             epsilon=1e-6,
             max_steps=100000,
             damping=0.5,
             spread_step=10,
             spread_percent=0.3,
             try_shrink=True):
        """
        Personalized PageRank with shrinking
        """

        assert (len(start_nodes)) > 0
        size = self.normalized.shape[0]
        if start_nodes is None:
            start_nodes = range(size)
            nz = size
        else:
            nz = len(start_nodes)
        start_vec = np.zeros((size, 1))
        start_vec[start_nodes] = 1
        start_rank = start_vec / len(start_nodes)
        rank_vec = start_vec / len(start_nodes)
        shrink = False
        which = np.zeros(0)
        if len(
                self.global_top_nodes
        ) < self.normalized.shape[1] and self.sample_whole_graph == False:
            which = np.full(self.normalized.shape[1], False)
            which[self.global_top_nodes] = True
            start_rank = start_rank[which]
            rank_vec = rank_vec[which]
            self.normalized = self.normalized[:, which][which, :]
            start_vec = start_vec[which]
            size = len(self.global_top_nodes)
        else:
            which = np.zeros(0)
        if try_shrink:
            v = start_vec / len(start_nodes)
            steps = 0
            while nz < size * spread_percent and steps < spread_step:
                steps += 1
                v += self.normalized.dot(v)
                nz_new = np.count_nonzero(v)
                if nz_new == nz:
                    shrink = True
                    break
                nz = nz_new
            rr = np.arange(self.normalized.shape[0])
            which = (v[rr] > 0).reshape(size)
            if shrink:
                start_rank = start_rank[which]
                rank_vec = rank_vec[which]
                self.normalized = self.normalized[:, which][which, :]
        diff = np.Inf
        steps = 0
        while diff > epsilon and steps < max_steps:
            steps += 1
            new_rank = self.normalized.dot(rank_vec)
            rank_sum = np.sum(new_rank)
            if rank_sum < 0.999999999:
                new_rank += start_rank * (1 - rank_sum)
            new_rank = damping * new_rank + (1 - damping) * start_rank
            new_diff = np.linalg.norm(rank_vec - new_rank, 1)
            diff = new_diff
            rank_vec = new_rank
        if try_shrink and shrink:
            ret = np.zeros(size)
            rank_vec = rank_vec.T[0]
            ret[which] = rank_vec
            if start_nodes[0] < len(ret):
                ret[start_nodes] = 0
            return ret.flatten()
        else:
            if start_nodes[0] < len(rank_vec):
                rank_vec[start_nodes] = 0
            return rank_vec.flatten()

    def emit(self, message):
        """
        Simple logging wrapper
        """
        logging.info(message)

    def get_sparse_walk_matrix(self,
                               num_important,
                               prob_threshold=0,
                               parallel_step=6):
        """
        Get walk matrix
        """

        if self.verbose:
            self.emit("Walking..")

        if self.node_names is None:
            self.node_names = list(range(self.input_graph.shape[1]))
        self.global_pagerank = sorted(self.sparse_pr().items(),
                                      key=operator.itemgetter(1),
                                      reverse=True)
        self.global_top_nodes = [
            x[0] for x in self.global_pagerank[0:num_important]
        ]
        self.stochastic_normalization()
        n = self.normalized.shape[1]
        edgelist_triplets = []
        jobs = [range(n)[i:i + parallel_step] for i in self.global_top_nodes]
        with mp.Pool(processes=parallel_step) as p:
            for batch in tqdm.tqdm(jobs):
                results = p.map(self.page_rank_kernel, batch)
                for nid, result_vector in results:
                    cols = np.argwhere(
                        result_vector > prob_threshold).flatten().astype(int)
                    vals = result_vector[cols].flatten()
                    ixx = np.repeat(nid, cols.shape[0]).flatten().astype(int)
                    arx = np.vstack((ixx, cols, vals)).T
                    edgelist_triplets.append(arx)
        sparse_edgelist = np.concatenate(edgelist_triplets, axis=0)
        print("Compressed to {}% of the initial size".format(
            (sparse_edgelist.shape[0] * 100) / n**2))
        vectors = sp.coo_matrix(
            (sparse_edgelist[:, 2], (sparse_edgelist[:, 0].astype(int),
                                     sparse_edgelist[:,
                                                     1].astype(int)))).tocsr()
        return vectors

    def approximate_normalized_graph_laplacian(self, A, rank, which="LA"): 
        
        n = A.shape[0]
        L, d_rt = csgraph.laplacian(A, normed=True, return_diag=True)
        
        X = sp.identity(n) - L
        self.emit("Eigen decomposition...")
        evals, evecs = eigsh(X, rank, which=which, sigma=0, tol=1E-2)
        self.emit("Maximum eigenvalue {}, minimum eigenvalue {}".format(
            np.max(evals), np.min(evals)))
        self.emit("Computing D^{-1/2}U..")
        D_rt_inv = sp.diags(d_rt**-1)
        D_rt_invU = D_rt_inv.dot(evecs)
        return evals, D_rt_invU
    
        """
        # NOTE: Swith to this version if encounter singular factor problem
        # Compute the graph Laplacian
        n = A.shape[0]
        L, d_rt = csgraph.laplacian(A, normed=True, return_diag=True)

        # Scale the Laplacian matrix to improve conditioning
        scaling_factor = np.max(np.abs(d_rt))  # Find the largest diagonal element
        scaled_L = L / scaling_factor  # Scale the Laplacian

        X = sp.identity(n) - scaled_L
        self.emit("Eigen decomposition...")
        evals, evecs = eigsh(X, rank, which=which, sigma=0, tol=1E-2)
        self.emit("Maximum eigenvalue {}, minimum eigenvalue {}".format(
            np.max(evals), np.min(evals)))
        self.emit("Computing D^{-1/2}U..")
        D_rt_inv = sp.diags(d_rt**-1)
        D_rt_invU = D_rt_inv.dot(evecs)
        return evals, D_rt_invU
        """

    def approximate_deepwalk_matrix(self, evals, D_rt_invU, window, vol, b):
        evals = self.deepwalk_filter(evals, window=window)
        X = sp.diags(np.sqrt(evals)).dot(D_rt_invU.T).T
        device = torch.device(
            self.device)  #('cuda' if torch.cuda.is_available() else 'cpu')
        m = torch.from_numpy(X).to(device)
        vx = torch.tensor(vol / b).to(device)
        mmt = torch.mm(m, m.t()).double()
        vol = vx.expand_as(mmt).double()
        mmt2 = mmt * vol
        Y = torch.log(torch.clamp(mmt2, min=1)).cpu()
        self.emit("Computed DeepWalk matrix with {} non-zero elements".format(
            np.count_nonzero(Y)))
        return sp.csr_matrix(Y)

    def svd_deepwalk_matrix(self, X, dim):
        self.emit("Computing SVD..")
        u, s, v = svds(X, dim, return_singular_vectors="u")
        return sp.diags(np.sqrt(s)).dot(u.T).T

    def deepwalk_filter(self, evals, window):
        for i in range(len(evals)):
            x = evals[i]
            evals[i] = 1. if x >= 1 else x * (1 - x**window) / (1 - x) / window
        evals = np.maximum(evals, 0)
        self.emit(
            "After filtering, max eigenvalue={}, min eigenvalue={}".format(
                np.max(evals), np.min(evals)))
        return evals

    def netMF_large(self,
                    A,
                    rank=256,
                    embedding_dimension=128,
                    window=10,
                    negative=1.0):

        self.emit("Running TorchNetMF for a large window size...")
        self.emit("Window size is set to be {}".format(window))

        if rank >= A.shape[1]:
            rank = A.shape[1] - 1

        #if embedding_dimension >= A.shape[1]:
        if embedding_dimension >= rank:
            embedding_dimension = rank

        # load adjacency matrix
        vol = float(A.sum())

        # perform eigen-decomposition of D^{-1/2} A D^{-1/2}
        # keep top #rank eigenpairs

        evals, D_rt_invU = self.approximate_normalized_graph_laplacian(
            A, rank=rank, which="LA")

        # approximate deepwalk matrix
        deepwalk_matrix = self.approximate_deepwalk_matrix(evals,
                                                           D_rt_invU,
                                                           window=window,
                                                           vol=vol,
                                                           b=negative)

        # factorize deepwalk matrix with SVD
        return self.svd_deepwalk_matrix(deepwalk_matrix,
                                        dim=embedding_dimension)

    def cross_entropy(self, data, clusters):
        """
        The symmetric cross entropy is computed as follows:
        SymCE = - sum_{i} (P(i) + Q(i)(log(P(i))+log(Q(i))

        Idea:
        for each potential cluster:
        compute intra cluster cross entropy
        Min intra, max inter.
        """

        per_cluster = defaultdict(list)
        unique_clusters = set(clusters)
        clusters = np.array(clusters)
        internal_entropies = []
        for label in unique_clusters:
            indices = np.where(clusters == label)
            cmat = data[indices]
            flat_vals = np.array(cmat.todense().flatten())
            entropy_internal = -np.sum(
                np.multiply(flat_vals, np.log(flat_vals)))
            if np.isnan(entropy_internal):
                entropy_internal = 1
            internal_entropies.append(entropy_internal)
        joint_entropy = 1 / np.mean(internal_entropies)
        return joint_entropy

    def compute_cluster_quality(self,
                                clusters,
                                weight="weight",
                                optional_data=None):

        if self.clustering_measure == "euclidean":
            distances = cdist(clusters, clusters, 'euclidean')
            max_dist = np.max(distances)  #/np.mean(distances)
            return max_dist

        elif self.clustering_measure == "modularity":
            return self.modularity(clusters, weight)

        elif self.clustering_measure == "silhouette":
            return silhouette_score(optional_data, clusters)

        elif self.clustering_measure == "davies_bouldin":
            return davies_bouldin_score(optional_data, clusters)

        elif self.clustering_measure == "calinski_harabasz":
            return calinski_harabasz_score(optional_data, clusters)

        elif self.clustering_measure == "entropy":
            return self.cross_entropy(optional_data, clusters)

        else:
            self.emit("Please, select a quality measure!")

    def kmeans_clustering(self,
                          vectors,
                          community_range,
                          stopping,
                          improvement_step=0.001,
                          return_score=False):

        if self.verbose:
            self.emit("Doing kmeans search")

        if len(community_range) == 1:
            fine_interval = 0

        else:
            fine_interval = int((community_range[2] - 1) / 2)
            community_range = np.arange(community_range[0], community_range[1],
                                        community_range[2])

        nopt = 0
        lag_num = 0
        mx_opt = 0
        opt_partitions = None
        fine_range = None
        all_scores = []

        for nclust in community_range:
            dx_hc = defaultdict(list)
            clustering_algorithm = MiniBatchKMeans(n_clusters=nclust,
                                                   init_size=3 * nclust,n_init="auto")
            clusters = clustering_algorithm.fit_predict(vectors).tolist()
            for a, b in zip(clusters, self.node_names):
                dx_hc[a].append(b)
            partitions = dx_hc
            lx = np.max([len(y) for x, y in partitions.items()])
            if self.clustering_measure in ["silhouette", "davies_bouldin", "calinski_harabasz", "entropy"]:
                mx = self.compute_cluster_quality(clusters,
                                                  optional_data=vectors)
            else:
                mx = self.compute_cluster_quality(
                    clustering_algorithm.cluster_centers_)
            self.cluster_quality[str(nclust)] = mx
            all_scores.append((nclust, mx))
            if mx > mx_opt + improvement_step:
                lag_num = 0
                opt_partitions = partitions
                nopt = nclust
                if self.verbose:
                    self.emit(
                        "Improved quality: {}, found {} communities. Maximum size {}"
                        .format(np.round(mx, 3), nopt, lx))
                mx_opt = mx
                self.opt_clust = partitions

                #save for later use
                self.clustering_algorithm = clustering_algorithm
                self.nclust = nclust
                self.vectors = vectors
                self.clusters = clusters

            else:
                lag_num += 1
                if self.verbose:
                    self.emit(
                        "No improvement for {} iterations".format(lag_num))
            if lag_num == stopping:
                if self.verbose:
                    self.emit("Stopping criterion reached. Fine tunning.")
                    break
                lag_num += 1

        if self.verbose:
            self.emit("Initial screening returned optimum of: {}".format(nopt))

        ## Do some fine tunning of the k
        if fine_interval > 0:
            fine_range = [
                x for x in np.arange(nopt - fine_interval, nopt +
                                     fine_interval, 1) if x > 0
            ]
            for cand in tqdm.tqdm(fine_range):
                dx_hc = defaultdict(list)
                clustering_algorithm = MiniBatchKMeans(n_clusters=nclust,
                                                       init_size=3 * nclust,n_init="auto")
                clusters = clustering_algorithm.fit_predict(vectors).tolist()
                for a, b in zip(clusters, self.node_names):
                    dx_hc[a].append(b)
                partitions = dx_hc
                if self.clustering_measure in ["silhouette", "davies_bouldin", "calinski_harabasz", "entropy"]:
                    mx = self.compute_cluster_quality(clusters,
                                                      optional_data=vectors)
                else:
                    mx = self.compute_cluster_quality(
                        clustering_algorithm.cluster_centers_)
                if mx > mx_opt + improvement_step:
                    lag_num = 0
                    opt_partitions = partitions
                    if self.verbose:
                        self.emit(
                            "Improved quality: {}, found {} communities.".
                            format(np.round(mx, 2), len(partitions)))
                    mx_opt = mx
                    self.opt_clust = dx_hc
                    nopt = nclust

        if return_score:
            return opt_partitions, mx_opt, all_scores

        else:
            return opt_partitions

    def get_exp_features(self, cutoff=0.01, order=2):

        core_graph = self.input_graph.tocsc()
        feature_matrices = [core_graph]
        for j in range(order):
            if self.verbose:
                self.emit("Order of computation: {}".format(j))
            new = sp.linalg.expm(feature_matrices[j])
            feature_matrices.append(new)
        final_matrix = feature_matrices[0]
        for en, j in enumerate(feature_matrices):
            if en > 0:
                tar = self.stochastic_normalization(j.tocsr(),
                                                    return_matrix=True)
                final_matrix = final_matrix.tocsr().multiply(tar)
        return final_matrix.tocsr()

    def list_arguments(self):
        argument_list = {
            "verbose": False,
            "sparisfy": True,
            "parallel_step": 6,
            "prob_threshold (only for ppr_embedding)": 0.0005,
            "community_range": "auto",
            "num_important (only for PPR)": 100,
            "clustering_measure": "silhouette",
            "stopping": 5,
            "improvement_step": 0.05,
            "node_feature_type (netmf_embedding or ppr_embedding)":
            "netmf_embedding",
            "negative_range (negative sampling range)": [1],
            "window_sizes (range of netmf window sizes)": [3],
            "dims (latent dimension for netmf)": [64],
            "output_format (grouped or nongrouped)": "nongrouped",
            "use_normalized_scores (bool, normalize per netmf hyperparams)":
            True
        }
        for k, v in argument_list.items():
            print(k, "default setting:", v)

    def detect_communities(self,
                           verbose=False,
                           sparisfy=True,
                           parallel_step=6,
                           prob_threshold=0.0005,
                           community_range="auto",
                           num_important=100,
                           clustering_measure="silhouette",
                           stopping=5,
                           improvement_step=0.05,
                           node_feature_type="netmf_embedding",
                           negative_range=[1],
                           window_sizes=[3],
                           dims=[64],
                           output_format="nongrouped",
                           use_normalized_scores=True,
                           custom_embedding_vectors=None):

        if community_range == "auto":
            K = self.input_graph.shape[1]
            kpow = int(0.42 * np.power(K, 2 / 3) - 5.7)
            community_range = [kpow, K, kpow]
            self.emit(f"Estimated range to be explored: {community_range}")

        if self.verbose:
            self.emit("Important nodes: {}".format(num_important))
        self.clustering_measure = clustering_measure

        ## step 1: embedding
        self.sample_whole_graph = False
        if node_feature_type == "EXP":
            vectors = self.get_exp_features()
        elif node_feature_type == "netmf_embedding":

            ## optimize the joint space..
            lopt = 0
            best_partition = None
            self.opt_score = -1
            self.opt_k = -1
            self.opt_mean = -1
            all_scores = None
            self.opt_trace = []
            for n in negative_range:
                for w in window_sizes:
                    for d in dims:
                        if self.verbose:
                            self.emit("testing setting {} {} {}".format(
                                n, w, d))
                        vectors = self.netMF_large(self.input_graph,
                                                   embedding_dimension=d,
                                                   window=w,
                                                   negative=n)
                        tmp_partition, score, score_dump = self.kmeans_clustering(
                            vectors,
                            community_range,
                            stopping,
                            improvement_step,
                            return_score=True)
                        
                        if use_normalized_scores and len(score_dump) > 1:
                            normalized_score = (score - np.min(score_dump)) / (
                                np.max(score_dump) - np.min(score_dump))
                        else:
                            normalized_score = score

                        norm_score_dump = (score_dump - np.min(score_dump)) / (
                            np.max(score_dump) - np.min(score_dump))

                        score_mean = np.mean(
                            norm_score_dump
                        )  ## this was added after publication as it works better!
                        self.opt_trace.append({
                            "score": normalized_score,
                            "negative": n,
                            "window": w,
                            "dimension": d,
                            "smean": score_mean
                        })
                        if normalized_score > self.opt_score and tmp_partition and score_mean > self.opt_mean:
                            self.opt_mean = score_mean
                            self.all_scores = score_dump
                            self.opt_k = len(tmp_partition)
                            best_partition = tmp_partition
                            self.final_embedding = vectors
                            self.opt_score = normalized_score
                            self.opt_config = {
                                "score": normalized_score,
                                "negative": n,
                                "window": w,
                                "dimension": d,
                                "Num communities": len(tmp_partition),
                                "smean": score_mean
                            }
                        else:
                            self.emit("Invalid embedding")

        elif node_feature_type == "ppr_embedding":
            vectors = self.get_sparse_walk_matrix(num_important,
                                                  prob_threshold,
                                                  parallel_step)
            self.emit("Starting cluster detection..")
            best_partition, score, score_dump = self.kmeans_clustering(
                vectors,
                community_range,
                stopping,
                improvement_step,
                return_score=True)
            self.opt_score = score

        elif node_feature_type == "custom_embedding":
            self.emit("Starting cluster detection..")
            best_partition, score, score_dump = self.kmeans_clustering(
                custom_embedding_vectors,
                community_range,
                stopping,
                improvement_step,
                return_score=True)
            self.opt_score = score

        if self.verbose:
            self.emit("Obtained vectors of shape {}".format(vectors.shape))
            self.emit(self.opt_config)

        assert vectors.shape[0] == self.input_graph.shape[0]

        
        measures = ["silhouette", "davies_bouldin", "calinski_harabasz"]
        for m in measures:
            self.clustering_measure = m
            score = self.compute_cluster_quality(self.clusters, optional_data=self.vectors)
            self.quality_scores[m] = score
            self.emit("{} score: {}".format(m, score))
        
        if output_format == "grouped":
            return best_partition

        else:
            out_struct = {}
            for k, els in best_partition.items():
                for el in els:
                    out_struct[el] = k
            return out_struct

## FUNCTIONS TO READ INPUT FILES ##

In [ ]:
def cleandf(df):
    df = df.apply(lambda x: x.fillna(''))
    df = df.apply(lambda x: x.astype(str).str.lower())
    #df = df.apply(lambda x: x.astype(str).str.replace('\W', ' ', regex=True))
    df.replace('-', ' ', inplace=True) #replace all '-' values with NaN
    df.dropna(how="all", axis=1, inplace=True) #drop columns where all values are NaN
    return df

def extract(filename, encoding, sep):
    df = pd.read_csv(filename, sep, engine='python', encoding = encoding)
    return cleandf(df)

def get_field_words(stemmer, lemmatizer, dataset):    
    field_words = set()
    ignores = ['label', 'class']
    for c in dataset.columns:
        #ignore 'label' or 'class' column-name
        if c.strip().lower() in ignores:
            #print(c)
            continue        
        words = [s for s in re.split('\(|\)|-|_|,|;|\.|%|\*', '_'.join(c.split('\\')))]
        #print(f"===============: {words}")
        words = [re.sub('[0-9]', '', i) for i in words]
        words = [lemmatizer.lemmatize(w).upper() for w in words if len(w)>2 and w.isalnum()]
        field_words.update(words)
    return field_words

def infer_types(aPath, csv_infer):
    aprox_schema = csv_infer.run_inference(aPath)
    typs = []
    for k, v in aprox_schema.items():
        typs.append(v.get('type'))
    return typs

# Function to convert inferred column types to pandas dtype
def convert_to_dtype(inferred_type):
    if inferred_type == "INTEGER":
        return 'int64'
    elif inferred_type == "FLOAT":
        return 'float64'
    elif inferred_type in ["STRING", "DATE"]:
        return 'str'
    elif inferred_type == "DATETIME":
        return 'datetime64[ns]'
    elif inferred_type == "BOOLEAN":
        return 'bool'
    else:
        return 'str'  # Convert unknown types to string

def get_IQR(column):
    iqr=0
    try:
        q1 = column.quantile(0.25)
        q3 = column.quantile(0.75)
        iqr = q3 - q1
        return iqr
    except:
        return iqr

def get_sparsity(column):
    missing_values = column.isnull().sum()

    # Calculate the total number of values in the column
    total_values = column.count() + missing_values

    # Calculate the sparsity
    sparsity = (missing_values / total_values) * 100
    return sparsity
    
def extract_metadata(in_column):
    metadata_length = 10
    metadata = [0] * metadata_length

    column = in_column.dropna()

    # meta-features
    if column.dtype in ['int32', 'float32', 'int64', 'float64']:
        metadata[0] = column.min()
        metadata[1] = column.max()
        metadata[2] = column.mean()
        metadata[3] = column.median()
        metadata[4] = column.std()
        metadata[5] = column.skew()
        metadata[6] = get_sparsity(column)
    elif column.dtype in ['string', 'object', 'category']:
        metadata[7] = len(column.unique())
    elif column.dtype == 'datetime64':
        metadata[8] = column.min().timestamp()
        metadata[9] = column.max().timestamp()
    elif column.dtype == 'timedelta64':
        metadata[8] = column.min().total_seconds()
        metadata[9] = column.max().total_seconds()
    elif column.dtype == 'bool':
        metadata[2] = column.mean()

    # Convert the metadata list to a numpy array
    metadata = np.array(metadata).reshape(-1, 1)
    metadata = np.nan_to_num(metadata, nan=0.0)

    # Scale the metadata using MinMaxScaler
    #scaler = MinMaxScaler()
    scaler = StandardScaler()
    metadata = np.array(metadata).reshape(-1, 1)
    metadata = scaler.fit_transform(metadata)
    metadata = metadata.flatten().tolist()

    return metadata

def get_bins_data(df, n_components=10):
    # Initialize a dictionary to store the summarized metadata
    all_metadata = {}

    # Define the number of PCA components to keep
    dataset_metadata = []  # Store metadata for the current dataset    
    for col in df.columns:
        column = df[col]
        metadata = extract_metadata(column)  # The metadata contains a list of 10 bin values
        n_components = len(df.columns) if len(df.columns) < n_components else n_components
        dataset_metadata.extend(metadata)  # Extend the list with metadata values for this column

    # Reshape the metadata to a 2D array (rows = columns, columns = metadata values)
    dataset_metadata = np.array(dataset_metadata).reshape(len(df.columns), -1)

    # Standardize the metadata values
    scaler = StandardScaler()
    dataset_metadata = scaler.fit_transform(dataset_metadata)

    # Perform PCA to reduce dimensionality
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(dataset_metadata)

    # Store the PCA result in all_metadata
    final_metadata = pca_result.tolist()
    avg_pca_result = np.mean(final_metadata, axis=0)

    bins = np.linspace(min(avg_pca_result), max(avg_pca_result), n_components + 1)
    return np.digitize(avg_pca_result, bins) - 1

def get_dataset_metadata(dataset):
    dataset_metadata = []
    for col in dataset.columns:
        column = dataset[col]
        metadata = extract_metadata(column)
        dataset_metadata.extend(metadata)

    return np.array(dataset_metadata)

def read_it(datasets, directory, encoding, sep, max_size, stemmer, lemmatizer, csv_infer):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            pathfile = os.path.join(directory, filename)
            isCSV = filename.endswith('.csv')
            #print(isCSV)
            isFile = os.path.isfile(pathfile)
            #print(isFile)

            if not isCSV or not isFile:
                 datasets = read_it(datasets, pathfile, encoding, sep, max_size, stemmer, lemmatizer, csv_infer)
            else:
                #print(f"Reading File: [{filename}]...")
                file_size = os.path.getsize(pathfile)
                if (max_size != 0 and file_size > max_size):
                    #print(f"File Size is more than [{max_size}]. Skipped.")
                    continue
                    
                df = extract(pathfile, encoding, sep)
                names = get_field_words(stemmer, lemmatizer, df)
                if len(names)>0:
                    types = infer_types(pathfile, csv_infer)
                    metadata = get_dataset_metadata(df)
                    datasets[os.path.splitext(filename)[0]] = [names, types, pathfile, metadata] #0=names, 1=types, 2=pathfile, 3=metadata
                                        
    return datasets

## FUNTIONS FOR LSH - MINHASH ##

In [ ]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, size+1))
    my_random.shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocabs_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocabs_size))
    return hashes

def create_hash(vector: list, vocabs: list, minhash_func: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocabs)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

def probability(s, r, b):
    # s: similarity
    # r: rows (per band)
    # b: number of bands
    return 1 - (1 - s**r)**b

def get_zips(nbr_minhash_funcs, nbr_bands, dict_records):
    s_t = time.strftime("%H:%M:%S", time.localtime())
    start_time = time.perf_counter()
    print(f"Function:get_zips() started at {s_t}")
    
    #create vocabs
    vocabs = set()
    for idx, rec in dict_records.items():
        if len(rec)>0:
            vocabs.update(rec)
    vocabs = list(vocabs)
    
    # one-hot encoded the records
    encodeds = {}
    for idx, rec in dict_records.items():
        encodeds[idx] = [1 if x in rec else 0 for x in vocabs]
        
    #create hash functions
    minhash_func = build_minhash_func(len(vocabs), nbr_minhash_funcs)

    #create signatures
    signs = {}
    for idx, enc in encodeds.items():
        signs[idx] = create_hash(enc, vocabs, minhash_func)

    #create bands
    bands = {}
    for idx, sign in signs.items():
        #from signature with [nbr_minhash_funcs], split into [nbr_bands], each contain 5 rows
        bands[idx] = split_vector(sign, nbr_bands)
    
    finish_time = time.perf_counter()
    e_t = time.strftime("%H:%M:%S", time.localtime())
    print(f"Function:get_zips() finished at {e_t} in {(finish_time-start_time)/60} minutes")
    return list(combinations(bands.items(), 2))

def find_node(nodes: list, name: str):
    for idx, node in nodes.items():
        if name == node.name:
            return node
    return None

def get_nodes_edges(result, nodes, edges):
    for res_nodes, res_edge in result.get():
        ns = list(res_nodes.items())
        left = ns[0][0]
        right = ns[1][0]
        node_left = find_node(nodes, left)
        if (not node_left):
            nodes[left] = ns[0][1]
        
        node_right = find_node(nodes, right)
        if (not node_right):
            nodes[right] = ns[1][1]

        if (res_edge.values()):
            edges[list(res_edge.keys())[0]] = list(res_edge.values())[0]
            
#compute the Jaccard similarity to update the weight
def jaccard_set(list1, list2):
    """Define Jaccard Similarity function for two sets"""
    set1 = set(list1)
    set2 = set(list2)
    jaccard = len(set1.intersection(set2)) / len(set1.union(set2))
    simm = len(set1.symmetric_difference(set2))
    len_diff = max(simm / len(set1.union(set2)), (min(len(list1), len(list2))/max(len(list1), len(list2))))
    return jaccard*len_diff    
    
    #intersection = len(list(set(list1).intersection(list2)))
    #union = (len(list1) + len(list2)) - intersection
    #nList1 = len(list1)
    #nList2 = len(list2)
    #return float(intersection) / union * (min(nList1, nList2) / max(nList1, nList2))
    
def set_weight(edges, records, threshold=0.5):
    for k, edge in edges.items():
        curr_weight = edge.getWeight()
        left = edge.node_left.name
        left = list(records.get(left))
        right = edge.node_right.name
        right = list(records.get(right))
        new_weight = jaccard_set(left, right)
        weight = new_weight if new_weight > curr_weight else curr_weight
        #weight = new_weight if curr_weight <= 0.0 else (curr_weight + new_weight)/2
        if (weight) > threshold:
            edge.setWeight(weight)

## FUNCTIONS FOR LSH - RANDOM PROJECTIONS ##

In [ ]:
def pad_or_trim_data(original_data, n_components):
    if n_components < original_data.shape[0]:
        # Trim the data to the desired number of components
        padded_data = original_data[:n_components]
    elif n_components > original_data.shape[0]:
        # Calculate statistics from the original data
        mean = np.mean(original_data)
        std = np.std(original_data)
        min_val = np.min(original_data)
        max_val = np.max(original_data)

        # Number of dimensions to pad
        padding_dims = n_components - original_data.shape[0]

        # Generate synthetic data with similar distribution
        synthetic_data = []
        for i in range(padding_dims):
            # Generate synthetic values based on original data's statistics
            #synthetic_values = my_random.normal(loc=mean, scale=std)
            synthetic_values = my_random.gauss(mean, std)
            synthetic_values = np.clip(synthetic_values, min_val, max_val)  # Ensure values are within original range
            synthetic_data.append(synthetic_values)

        # Combine original and synthetic data
        padded_data = np.hstack((original_data, np.array(synthetic_data)))
    else:
        # No padding or trimming needed
        padded_data = original_data

    return padded_data

def get_largest_shape(dfs_metadata):
    largest_shape = 0
    for k, df_metadata in dfs_metadata.items():
        shape_0 = df_metadata.shape[0]
        if shape_0 > largest_shape:
            largest_shape = shape_0
    print(f"largest_shape == {largest_shape}")
    return largest_shape

def get_best_nbits(n_samples, buckets=[2, 4, 8, 16, 24, 32, 64]):
    best_nbits = 0
    prev_samples_buckets = np.inf
    for nbits in buckets:
        buckets = 1 << nbits
        samples_buckets = n_samples/buckets
        print(f"nbits == {nbits}")
        print(f"{n_samples} / {buckets} = {samples_buckets}, samples_buckets: {samples_buckets}, prev_samples_buckets:{prev_samples_buckets}")
        if samples_buckets > 0.1 and samples_buckets < prev_samples_buckets:
            prev_samples_buckets = samples_buckets
            best_nbits = nbits
        else:
            break
    
    print(f"best_nbits == {best_nbits}")
    return best_nbits


def convert_keys_to_indices(data_dict):
    indexed_dict = {}
    for idx, (key, value) in enumerate(data_dict.items()):
        indexed_dict[idx] = value
    return indexed_dict

def get_meta_edges(dfs_metadata_in):
    keys = list(dfs_metadata_in.keys())
    dfs_metadata = convert_keys_to_indices(dfs_metadata_in)
    
    largest_shape = get_largest_shape(dfs_metadata)
    nbits = get_best_nbits(len(dfs_metadata))

    d = largest_shape
    #plane_norms = my_random.rand(nbits, d) - .5
    plane_norms = np.array([[my_random.random() - 0.5 for _ in range(d)] for _ in range(nbits)])
    
    vectors = []
    padded_metadata = []
    for k, df_metadata in dfs_metadata.items():
        padded_meta = pad_or_trim_data(df_metadata, largest_shape)
        dot_data = np.dot(padded_meta, plane_norms.T)
        dot_data = dot_data > 0
        dot_data = dot_data.astype(int)
        vectors.append(dot_data)
        padded_metadata.append(padded_meta)
    
    buckets = {}
    i = 0
    for i in range(len(vectors)):
        # convert from array to string
        hash_str = ''.join(vectors[i].astype(str))
        # create bucket if it doesn't exist
        if hash_str not in buckets.keys():
            buckets[hash_str] = []
        # add vector position to bucket
        buckets[hash_str].append(i)

        
    edges = {}
    for k, v in buckets.items():
        combs = list(combinations(v, 2))
        for (l, r) in combs:
            left = dfs_metadata[l].reshape(1, -1)
            right = dfs_metadata[r].reshape(1, -1)
            padded_left = padded_metadata[l].reshape(1, -1)
            padded_right = padded_metadata[r].reshape(1, -1)
            score = cosine_similarity(padded_left, padded_right)
            edge = Edge(Node(keys[l]), Node(keys[r]), score[0, 0])
            edges[create_edge_key(edge)] = edge
                
    #for key, edge in edges.items():
    #    print(f"key:{key} => {edge}")
        
    return edges

## FUNCTIONS FOR PREPARATIONS ##

In [ ]:
def get_csv_infer():
    conditions={"INTEGER":"FLOAT"}
    return csv_schema_inference.CsvSchemaInference(portion=0.9, max_length=100, batch_size = 200000, acc = 0.8, seed=2, header=True, sep=",", conditions = conditions)

def read_files(directory, encoding='UTF-8', sep=',', max_size=0):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    csv_infer = get_csv_infer()
    
    datasets = {}
    datasets = read_it(datasets, directory, encoding, sep, max_size, stemmer, lemmatizer, csv_infer)
    return csv_infer, datasets

def unpack_datasets(datasets):
    # Initialize separate dictionaries for each variable
    names = {}
    types = {}
    pathfiles = {}
    dfs_metadata = {}

    # Iterate through the original dictionary
    for key, values in datasets.items():
        names[key] = values[0]
        types[key] = values[1]
        pathfiles[key] = values[2]
        dfs_metadata[key] = values[3]

    return names, types, pathfiles, dfs_metadata

def read_input(directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands):
    if fresh:
        csv_infer, datasets = read_files(directory, encoding=encoding, sep=sep, max_size=max_size)
        
        with open(directory+'datasets.pkl', 'wb') as fp:
            pickle.dump(datasets, fp)
            print('datasets saved successfully to file')
    
    else:
        csv_infer = get_csv_infer()
        
        # Read dictionary pkl file
        with open(directory+'datasets.pkl', 'rb') as fp:
            datasets = pickle.load(fp)
            print('file datasets.pkl read successfully')

    return csv_infer, datasets

def merge_recs(rec_dict1, rec_dict2):
    merged_recs = {}

    # Add nodes from the first dictionary
    for key, rec in rec_dict1.items():
        merged_recs[key] = rec

    # Add nodes from the second dictionary
    for key, rec in rec_dict2.items():
        if key not in merged_recs:
            merged_recs[key] = rec

    return merged_recs

def merge_nodes(node_dict1, node_dict2):
    merged_nodes = {}

    # Add nodes from the first dictionary
    for key, node in node_dict1.items():
        merged_nodes[key] = node

    # Add nodes from the second dictionary
    for key, node in node_dict2.items():
        if key not in merged_nodes:
            merged_nodes[key] = node

    return merged_nodes

def update_edges(edge_dict1, edge_dict2, replace_only=False):
    p_param = 0.5
    merged_edges = {}
    
    # Add edges from the first dictionary
    for key, edge in edge_dict1.items():
        merged_edges[key] = edge
    
    # Add edges from the second dictionary
    for key, edge in edge_dict2.items():
        if key not in merged_edges and not replace_only:
            merged_edges[key] = edge
        elif key in merged_edges:
            edge1 = merged_edges.get(key)
            weight1 = edge1.getWeight()
            edge2 = edge
            weight2 = edge2.getWeight()
            average_weight = weight1 if weight1 > weight2 else weight2
            if replace_only:
                weight2 = p_param*weight2
                average_weight = weight1 if weight1 > weight2 else weight2
            edge1.setWeight(average_weight)
            
    return merged_edges

def do_LSH(datasets, n_hash_fs, n_bands):
    nm_recs, ty_recs, pathfiles, dfs_metadata = unpack_datasets(datasets)
    nm_zips = get_zips(n_hash_fs, n_bands, nm_recs)
    ty_zips = get_zips(n_hash_fs, n_bands, ty_recs)
    
    s_t = time.strftime("%H:%M:%S", time.localtime())
    start_time = time.perf_counter()
    print(f"Function:LSH-based Similarity started at {s_t}")
    print(f"Processing total datasets: {len(datasets)}")

    nm_nodes = {}
    nm_edges = {}
    ty_nodes = {}
    ty_edges = {}

    with mp.Pool() as pool:
        result = pool.map_async(create_netw, nm_zips)
        get_nodes_edges(result, nm_nodes, nm_edges)

    with mp.Pool() as pool:
        result = pool.map_async(create_netw, ty_zips)
        get_nodes_edges(result, ty_nodes, ty_edges)

    finish_time = time.perf_counter()
    e_t = time.strftime("%H:%M:%S", time.localtime())
    print(f"Function:LSH-based Similarity finished at {e_t} in {(finish_time-start_time)/60} minutes")
    
    set_weight(nm_edges, nm_recs)
    set_weight(ty_edges, ty_recs)
    
    meta_edges = get_meta_edges(dfs_metadata)
    
    nodes = merge_nodes(nm_nodes, ty_nodes)
    edges = update_edges(nm_edges, ty_edges)
    edges = update_edges(ty_edges, meta_edges, replace_only=True)
    
    print(f"Number of nodes: {len(nodes)}")
    print(f"Number of edges: {len(edges)}")

    return nm_nodes, nm_edges, nodes, edges

def prepare(directory, fresh, encoding='ISO-8859-1', sep=',', max_size=500000, n_hash_fs=100, n_bands = 20):
    csv_infer, datasets = read_input(directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands)
    ini_nodes, init_edges, merge_nodes, merge_edges = do_LSH(datasets, n_hash_fs, n_bands)

    #for k, edge in edges.items():
    #    print(edge)
    #    break
    
    return csv_infer, datasets, ini_nodes, init_edges, merge_nodes, merge_edges

def prepare_all(root_dirs, fresh, encoding='ISO-8859-1', sep=',', max_size=500000, n_hash_fs=100, n_bands = 20):
    all_datasets = {}
    csv_infer = None
    
    for k, values in root_dirs.items():
        directory = values[0]
        max_size = values[1]
        csv_infer, datasets = read_input(directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands)
        print(f"Adding {k} with datasets: {len(datasets)}")
        all_datasets.update(datasets)
        
    ini_nodes, init_edges, merge_nodes, merge_edges = do_LSH(all_datasets, n_hash_fs, n_bands)
    
    return csv_infer, all_datasets, ini_nodes, init_edges, merge_nodes, merge_edges

## GRAPH NETWORK ##

In [ ]:
def get_graph(nodes, edges, merge, threshold=0.5):
    G = nx.Graph()

    for k, node in nodes.items():
        G.add_node(node.name)

    for k, edge in edges.items():
        w = edge.edge_weight
        if w > threshold:
            a = edge.node_left
            b = edge.node_right
            G.add_edge(a.name, b.name, weight=w)
            
    if merge:
        # Step 1: Remove node with degree < 2
        nodes_to_remove = [node for node in G.nodes() if G.degree(node) < 2]
        G.remove_nodes_from(nodes_to_remove)
        
        # Step 2: Find connected components in the modified graph
        connected_cs = list(nx.connected_components(G))

        # Step 3: Identify the connected component with the least number of members
        min_size_component = min(connected_cs, key=len)

        # Step 4: Add the removed nodes to the identified connected component
        last_element = None
        for node in nodes_to_remove:
            G.add_node(node)  # Add the node
            if min_size_component:
                last_element = min_size_component.pop()
                G.add_edge(node, last_element, weight=0.5)  # Add an edge to the chosen component
            elif last_element:
                G.add_edge(node, last_element, weight=0.5)  # Use the last element for the remaining nodes

        # Step 1: Remove node with degree < 2
        nodes_to_remove = [node for node in G.nodes() if G.degree(node) < 2]
        G.remove_nodes_from(nodes_to_remove)
        
        # Step2: Find connected components with members less than 5
        #small_components = [component for component in nx.connected_components(G) if len(component) < 5]

        # Step 3: Remove nodes from small connected components
        #nodes_to_remove = [node for component in small_components for node in component]
        #G.remove_nodes_from(nodes_to_remove)
        
    return G

def get_communities(G, nodes):
    sparse_mat = nx.to_scipy_sparse_matrix(G)
    node_names = list(range(sparse_mat.shape[1]))
    SCD_detector = SCD_obj(sparse_mat,node_names=node_names)

    ## all hyperparameters
    SCD_detector.list_arguments()

    ## set hyperparameters
    param2 = {"verbose":True,"parallel_step":8,"stopping":25,"use_normalized_scores":False,"community_range":[2,len(G.nodes()),2]}
    communities = SCD_detector.detect_communities(**param2)
    
    #communities = nx.community.greedy_modularity_communities(G, weight="weight")
    # Create a dictionary where each node is assigned to its own community
    #node_to_community = {node: i for i, component in enumerate(nx.connected_components(G)) for node in component}
    # Convert the dictionary to a list of communities
    #communities = [set() for _ in range(max(node_to_community.values()) + 1)]
    #for node, community in node_to_community.items():
    #    communities[community].add(node)
    
    # Create a new dictionary to store the grouped keys
    comms = {}

    # Iterate through the original dictionary
    for key, value in communities.items():
        if value in comms:
            comms[value].append(key)
        else:
            comms[value] = [key]
    #print(f"Communities before merging process: {comms}")
            
    # Create a new dictionary to store merged results
    merged_comms = {}    
    key_counter = 0  # Counter for assigning new keys
    for key, values in comms.items():
        if len(values) < 2 and merged_comms:
            # Find the key with the larger list to merge into
            max_key = max(merged_comms.keys(), key=lambda k: len(merged_comms[k]))
            # Merge the values into the key with the larger list
            merged_comms[max_key].extend(values)
        else:
            # If list has 5 or more elements, add to merged_data as is
            merged_comms[key_counter] = values
            key_counter += 1
    
    #print(f"Communities after merging process: {merged_comms}")
    
    node_indices = {index: node_name for index, node_name in enumerate(G.nodes())}
    final_communities = {}
    for key, indices in merged_comms.items():
        final_communities[key] = [{node_indices[index]: nodes[node_indices[index]]} for index in indices]
    
    #print(f"Final Communities: {final_communities}")
    
    return final_communities

def convert(comm):
    # Create a dictionary to store items grouped by their values
    grouped_dict = {}
    for key, value in comm.items():
        if value in grouped_dict:
            grouped_dict[value][key] = value
        else:
            grouped_dict[value] = {key: value}

    return list(grouped_dict.values())

def compute_metrics(other_methods, directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands, root_dirs=False):
    if root_dirs:
        _, _, _, _, nodes, edges = prepare_all(directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands)  
    else:
        _, _, _, _, nodes, edges = prepare(directory, fresh, encoding, sep, max_size, n_hash_fs, n_bands)  
    G = get_graph(nodes, edges, True)
    #pickle.dump(G, open(f'{directory}\\graph.pickle', 'wb'))
    
    sparse_mat = nx.to_scipy_sparse_matrix(G)
    node_names = list(range(sparse_mat.shape[1]))
    SCD_detector = SCD_obj(sparse_mat,node_names=node_names)
    SCD_detector.list_arguments()
    param2 = {"verbose":True,"parallel_step":8,"stopping":25,"use_normalized_scores":False,"community_range":[2,len(G.nodes()),2]}
    communities = SCD_detector.detect_communities(**param2)
    #print(f"communities:{communities}")
    modularities = {}
    if other_methods:
        modularities['OUR'] = SCD_detector.modularity(convert(communities))
        for m in other_methods:
            if m == "GMC":
                gmc_comm = nx.community.greedy_modularity_communities(G)            
                #print(f"gmc_comm:\n{gmc_comm}")
                gmc = nx.community.modularity(G, gmc_comm)
                modularities[m] = gmc
            elif m == "LPA":
                lpa_comm = nx.algorithms.community.label_propagation_communities(G)
                #print(f"lpa_comm:\n{lpa_comm}")
                lpa = nx.community.modularity(G, lpa_comm)
                modularities[m] = lpa
            elif m == "LVA":
                lva_comm = nx.algorithms.community.louvain_communities(G)
                #print(f"lva_comm:\n{lva_comm}")
                lva = nx.community.modularity(G, lva_comm)
                modularities[m] = lva

    return SCD_detector.quality_scores, modularities

## DRAW NETWORK ##

In [ ]:
def draw(nodes, edges, merge=False, threshold=0.5, spaces=0.15, chart_size=[5,5], legend_pos=[0.95, 0.3, 0.03, 0.3], 
         legend_label='Connected Components', remarks_pos=[1.3, 0.55, 0.03, 0.3]):
    G = get_graph(nodes, edges, merge, threshold)

    pos = nx.spring_layout(G, k=spaces)
    wcc = list(nx.connected_components(G))

    # Set the number of colors
    num_colors = len(wcc)
    # Generate a set of visually distinct colors using seaborn's color palette
    distinct_colors = sns.color_palette("husl", n_colors=num_colors)
    # Convert the colors to hexadecimal format for Networkx
    distinct_colors_hex = ['#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255)) for r, g, b in distinct_colors]
    
    
    thick = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= 0.9]
    medium = [(u, v) for (u, v, d) in G.edges(data=True) if 0.75 < d["weight"] < 0.9]
    thin = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.75]
    
    fig, ax = plt.subplots(figsize=chart_size)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')

    for index, sg in enumerate(wcc):
        nx.draw_networkx_nodes(sg, pos=pos, node_size=75, edgecolors="black", node_color=distinct_colors_hex[index])
        nx.draw_networkx_edges(G, pos=pos, ax=ax, edgelist=thick, width=2, edge_color="lightblue")        
        nx.draw_networkx_edges(G, pos=pos, ax=ax, edgelist=medium, width=1, edge_color="lightgreen")
        nx.draw_networkx_edges(G, pos=pos, ax=ax, edgelist=thin, width=1, alpha=0.5, edge_color="lightgrey", style="dashed")
        
        
    # add legends
    # https://matplotlib.org/stable/tutorials/colors/colorbar_only.html
    ax2 = fig.add_axes(legend_pos) # distance left, distance up, width size, height size
    cmap = (colors.ListedColormap(distinct_colors_hex))
    bounds = list(range(0, num_colors+1))
    norm = colors.BoundaryNorm(bounds, cmap.N)
    cbar = fig.colorbar(
        plt.cm.ScalarMappable(cmap=cmap, norm=norm),
        cax=ax2,
        extend='neither',
        ticks=bounds,
        spacing='proportional',
        orientation='vertical',

    )
    #cbar.ax.tick_params(labelsize=12)
    # Hide tick labels while keeping the boundary lines
    cbar.set_ticklabels([])
    ax2.set_ylabel(legend_label, size=12)

    ax3 = fig.add_axes(remarks_pos) # distance left, distance up, width size, height size
    ax3.axis("off")
    
    custom_lines = [Line2D([0], [0], color="lightblue", lw=2),
                    Line2D([0], [0], color="lightgreen", lw=1),
                    Line2D([0], [0], color="lightgrey", lw=1, linestyle='dashed',)]

    ax3.legend(custom_lines, ['feature_similarity >= 0.9', '0.75 < feature_similarity < 0.9', 'feature_similarity <= 0.75'],
               loc='best', prop={'size': 10}, fancybox=True, framealpha=0)

    #plt.savefig('Figure_node_lines.png', bbox_inches='tight')
    plt.show()
    
    return G

def draw_net(ini_nodes, init_edges, merge_nodes, merge_edges, threshold=0.5, spaces=0.2, chart_size=[8,6], 
     legend_pos=[0.95, 0.15, 0.03, 0.5], legend_label1='Network Nodes & Edges', legend_label2='Connected Components', remarks_pos=[1.25, 0.55, 0.03, 0.3]):
    results = pd.DataFrame({
        's': [],
        'P': [],
        'r,b': []
    })

    for s in np.arange(0.01, 1, 0.01):
        total = 100
        factors = []
        for i in range(1, total+1):
            if total%i==0:
                factors.append(i)
        for b in factors:
            r = int(total/b)
            P = probability(s, r, b)
            results = results.append({
                's': s,
                'P': P,
                'r,b': f"{r},{b}"
            }, ignore_index=True)

    sns.lineplot(data=results, x='s', y='P', hue='r,b')
        
    draw(ini_nodes, init_edges, threshold=threshold, spaces=spaces, chart_size=chart_size, legend_pos=legend_pos, legend_label=legend_label1, remarks_pos=remarks_pos)
    G = draw(merge_nodes, merge_edges, merge=True, threshold=threshold, spaces=spaces, chart_size=chart_size, legend_pos=legend_pos, legend_label=legend_label1, remarks_pos=remarks_pos)
    communities = get_communities(G, merge_nodes)
    
    # create a list of colors from a matplotlib colormap
    cmap = matplotlib.cm.get_cmap('tab20')

    color_intervals = np.linspace(0,1,len(list(communities)))
    colors_tab = []
    for value in color_intervals:
        colors_tab.append(cmap(value))

    # add color to nodes by community group
    community_color_map = []
    for node in G.nodes:
        for idx, community_list in communities.items():
            community_keys = [list(comm_dict.keys())[0] for comm_dict in community_list]
            #print(f'idx:{idx}, comm:{community_keys}')
            if node in community_keys:
                community_color_map.append(colors_tab[idx])

    pos = nx.spring_layout(G, k=0.5) # added a seed for layout reproducibility
    fig, (ax_data, ax_legend) = plt.subplots(1, 2, figsize=(8, 5), gridspec_kw={'width_ratios': [4, 1]})

    # Create a legend for communities
    legend_labels = [f"Community {idx}" for idx in range(len(communities))]
    legend_handles = [matplotlib.patches.Patch(color=colors_tab[idx], label=label) for idx, label in enumerate(legend_labels)]

    # Position the legend outside the chart area
    ax_legend.legend(handles=legend_handles, bbox_to_anchor=(1, 1), loc='upper right')
    ax_legend.axis("off")

    # Add a border around the entire figure
    border_color = 'grey'  # Color of the border
    border_width = 0.25  # Width of the border
    fig.patch.set_edgecolor(border_color)
    fig.patch.set_linewidth(border_width)

    # Draw the networkx graph inside the ax_data subplot
    nx.draw(G, pos, ax=ax_data, node_color=community_color_map, edge_color="lightgrey", node_size=50, edgecolors='black', linewidths=0.25)

    plt.tight_layout()
    plt.show()
    
    #print(f"Modularity: {round(nx_comm.modularity(G, communities),3)}")
    
    return G, communities
        
def draw_all(csv_infer, datasets, ini_nodes, init_edges, merge_nodes, merge_edges, encoding='ISO-8859-1', sep=',', include_metadata=True, max_metadata_length=100, threshold=0.5, spaces=0.2, chart_size=[8,6], 
     legend_pos=[0.95, 0.15, 0.03, 0.5], legend_label1='Network Nodes & Edges', legend_label2='Connected Components', remarks_pos=[1.25, 0.55, 0.03, 0.3]):

    #drwa network
    G, communities = draw_net(ini_nodes, init_edges, merge_nodes, merge_edges, threshold, spaces, chart_size, legend_pos, legend_label1, legend_label2, remarks_pos)
    
    #create classifier model
    untrained_model, community_labels, feature_vocab, X, y = build_classifier(csv_infer, datasets, G, communities, encoding, sep, include_metadata, max_metadata_length)
    
    #train k-fold
    trained_kfold_model = train_kfold(community_labels, X, y)
    
    #train shuffle
    trained_shuffle_model = train_shuffle(community_labels, X, y)
    
    return untrained_model, trained_kfold_model, trained_shuffle_model, communities, community_labels, feature_vocab, X, y    

## CLASSIFIER ##

In [ ]:
# Define a model
class CommunityClassifier(nn.Module):
    def __init__(self, in_feats, num_classes):
        super(CommunityClassifier, self).__init__()
        self.fc = nn.Linear(in_feats, num_classes)
    
    def forward(self, x):
        x = self.fc(x)
        return x

def build_classifier(csv_infer, datasets, G, communities, encoding, sep, include_metadata, max_metadata_length):
    # Create numerical community labels
    community_labels = list(range(len(communities)))
    # Print the numerical community labels to verify
    #print(community_labels)

    n_community_labels = torch.tensor(community_labels, dtype=torch.long)
    #print(n_community_labels)

    # Define a LabelEncoder for mapping categorical features to numerical values
    label_encoder = LabelEncoder()
    
    # Unpack datasets
    names, types, pathfiles, dfs_metadata = unpack_datasets(datasets)

    community_features = []
    for idx, community_list in communities.items():
        community_keys = [list(comm_dict.keys())[0] for comm_dict in community_list]
        c_feats = []
        for c in community_keys:
            nm = list(names[c])
            typ = list(set(types[c]))
            feature = nm + typ
            c_feats.append({c:feature})
        community_features.append({idx: c_feats})

    #print(community_features)

    # Extract features and labels
    all_features = []
    all_labels = []
    all_id_feature_mappings = {}    
    
    flat_features = []
    for community_id, features_list in enumerate(community_features):
        for feature_dict in features_list.values():
            flat_features.extend((community_id, node_name, node_features) for node_info in feature_dict for node_name, node_features in node_info.items())

    all_features = [node_features for _, _, node_features in flat_features]
    all_labels = [community_id for community_id, _, _ in flat_features]
    all_id_feature_mappings = {idx: {node_name: node_features} for idx, (_, node_name, node_features) in enumerate(flat_features)}

    # Convert community labels to numerical values using LabelEncoder
    label_encoder = LabelEncoder()
    numerical_community_labels = label_encoder.fit_transform(all_labels)

    # Convert features to a format suitable for training
    feature_mapping = defaultdict(list)
    for idx, features in enumerate(all_features):
        feature_mapping[numerical_community_labels[idx]].extend(features)

    # Create a vocabulary of unique features
    unique_features = list(set(feature for features in feature_mapping.values() for feature in features))
    feature_vocab = {feature: idx for idx, feature in enumerate(unique_features)}

    # Convert feature lists to binary vectors using the vocabulary
    binary_features = []
    for features in all_features:
        binary_features.append([1 if feature in features else 0 for feature in feature_vocab])

    # Convert to PyTorch tensors
    X = torch.tensor(binary_features, dtype=torch.float)
    #print(len(all_features))
    #print(X.shape)
    
    if include_metadata:
        metadata_tensors = []
        for k, fp in pathfiles.items():
            if (k not in G.nodes):
                #it was pruned, skipped
                continue

            inferred_column_types = infer_types(fp, csv_infer)

            # Read the CSV file
            df = pd.read_csv(fp, sep, engine='python', encoding = encoding)
            df = cleandf(df)

            # Convert inferred column types to pandas dtypes
            column_types = [convert_to_dtype(inferred_type) for inferred_type in inferred_column_types]

            # Set the column types of the DataFrame with preprocessing for non-numeric values
            for col_name, dtype in zip(df.columns, column_types):
                if dtype in ('int64', 'float64'):
                    # Handle non-numeric values by replacing them with NaN
                    df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
                    df[col_name].fillna(0, inplace=True)  # Replace NaN with 0
                    df[col_name] = df[col_name].astype('float64')  # Convert to nullable float type
                else:
                    df[col_name] = df[col_name].astype(dtype)

            # Set the column types of the DataFrame
            df = df.astype(dict(zip(df.columns, column_types)))

            # Create a fixed-length tensor of metadata
            metadata_tensor = []
            for col_name in df.columns:
                column = df[col_name]
                metadata = extract_metadata(column)
                metadata_tensor.extend(metadata)

            # Convert the metadata list to a PyTorch tensor
            metadata_tensors.append(torch.tensor(metadata_tensor, dtype=torch.float))

        # Pad each metadata tensor to match the maximum length
        padded_metadata_tensors = []
        for metadata_tensor in metadata_tensors:
            padding_length = max_metadata_length - metadata_tensor.shape[0]
            if padding_length < 0:
                padded_metadata_tensor = metadata_tensor[:max_metadata_length]
            else:
                padded_metadata_tensor = torch.cat((metadata_tensor, torch.zeros(padding_length)))
            padded_metadata_tensors.append(padded_metadata_tensor)

        # Convert the list of padded metadata tensors to a single tensor
        final_metadata_tensor = torch.stack(padded_metadata_tensors)

        # Concatenate the feature tensor and final metadata tensor along the second dimension
        X = torch.cat((X, final_metadata_tensor), dim=1)
        
    y = torch.tensor(numerical_community_labels, dtype=torch.long)
    #print(f'X.shape:{X.shape}')
    #print(f'y.shape:{y.shape}')

    # Define parameters
    #in_feats = len(feature_vocab)  # Number of unique features
    in_feats = X.shape[1]
    num_classes = len(community_labels)  # Number of communities
    #print(f'in_feats:{in_feats}, num_classes:{num_classes}')

    # Initialize the model
    model = CommunityClassifier(in_feats, num_classes)
    
    return model, community_labels, feature_vocab, X, y

def train_kfold(community_labels, X, y, random_state=42):
    in_feats = X.shape[1]
    num_classes = len(community_labels)
    
    # Create a StratifiedKFold instance with the desired number of folds
    n_splits = 5  # Number of folds
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Lists to store evaluation metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Perform Stratified K-Fold cross-validation
    for fold, (train_idx, test_idx) in enumerate(stratified_kfold.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Initialize and train the model
        # Convert numerical community labels to string class names
        class_names = [str(label) for label in community_labels]
        #print(f'len(class_names): {len(class_names)}')

        # Define model, loss function, and optimizer
        model = CommunityClassifier(in_feats, num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Training loop
        for epoch in range(1000):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            test_output = model(X_test)
            predicted_labels = torch.argmax(test_output, dim=1)

        # Calculate evaluation metrics for the fold
        accuracy = accuracy_score(y_test, predicted_labels)
        precision = precision_score(y_test, predicted_labels, average='weighted')
        recall = recall_score(y_test, predicted_labels, average='weighted')
        f1 = f1_score(y_test, predicted_labels, average='weighted')

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold+1} - Accuracy: {accuracy:.5f}, Precision: {precision:.5f}, Recall: {recall:.5f}, F1-Score: {f1:.5f}")

        # Calculate the average metrics across all folds
        avg_accuracy = sum(accuracy_scores) / n_splits
        avg_precision = sum(precision_scores) / n_splits
        avg_recall = sum(recall_scores) / n_splits
        avg_f1 = sum(f1_scores) / n_splits

        print("\nAverage Metrics Across Folds:")
        print(f"Average Accuracy: {avg_accuracy:.5f}")
        print(f"Average Precision: {avg_precision:.5f}")
        print(f"Average Recall: {avg_recall:.5f}")
        print(f"Average F1-Score: {avg_f1:.5f}")
        
    return model

def train_shuffle(community_labels, X, y, test_size=0.3, random_state=42):
    in_feats = X.shape[1]
    num_classes = len(community_labels)
    
    # Create an instance of StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=5, test_size=test_size, random_state=random_state)

    # Split the data into train and test sets
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    # Convert numerical community labels to string class names
    class_names = [str(label) for label in community_labels]
    #print(f'len(class_names): {len(class_names)}')

    # Define model, loss function, and optimizer
    model = CommunityClassifier(in_feats, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    # Evaluate the trained model on the test set
    model.eval()
    with torch.no_grad():
        test_output = model(X_test)
        predicted_labels = torch.argmax(test_output, dim=1)

    accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted')
    recall = recall_score(y_test, predicted_labels, average='weighted')
    f1 = f1_score(y_test, predicted_labels, average='weighted')
    conf_matrix = confusion_matrix(y_test, predicted_labels)
    class_report = classification_report(y_test, predicted_labels, target_names=class_names)

    # Print evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(class_report)
    
    # Compute ROC curve and ROC AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, test_output[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Print ROC AUC for each class
    for i in range(num_classes):
        print(f"ROC AUC for Community {class_names[i]}: {roc_auc[i]:.4f}")
        
        
    # Print confusion matrix
    print("Confusion Matrix:")
    print(conf_matrix)

    # Display confusion matrix as a heatmap
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1)
    ConfusionMatrixDisplay(conf_matrix, display_labels=class_names).plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

    # Plot ROC Curves for each class
    # Plot ROC Curves for each class
    class_labels = [f"ROC curve (Community {class_names[i]})" for i in range(num_classes)]

    # Create a figure with two subplots side by side
    fig, (ax_data, ax_legend) = plt.subplots(1, 2, figsize=(8, 5), gridspec_kw={'width_ratios': [4, 1]})

    # Plot ROC Curves for each class on the ax_data
    legend_handles = []
    for i in range(num_classes):
        ax_data.plot(fpr[i], tpr[i])
        # Create custom legend handles for the ROC curves
        legend_handles.append(Line2D([0], [0], color='C{}'.format(i), lw=2))

    ax_data.plot([0, 1], [0, 1], 'k--', label='Random')
    ax_data.set_xlabel('False Positive Rate')
    ax_data.set_ylabel('True Positive Rate')
    ax_data.set_title('ROC Curves for Each Community')

    # Position the legend in the right subplot
    ax_legend.legend(handles=legend_handles, labels=class_labels, bbox_to_anchor=(1, 1), loc='upper right')
    ax_legend.axis("off")

    # Add a border around the entire figure
    border_color = 'grey'  # Color of the border
    border_width = 0.25  # Width of the border
    fig.patch.set_edgecolor(border_color)
    fig.patch.set_linewidth(border_width)

    plt.tight_layout()
    plt.show()
    
    return model